# Requirements

In [27]:
import itertools
import random
import warnings

# Problem

Consider a "map", i.e., a matrix of values of 0 and 1.  0 represents water, 1 represents land.  Islands are formed by elements that have the value 1 and that touch horizontally or vertically, not diagonally.

We start by writing a function to create a map.

In [14]:
def create_map(nr_rows, nr_cols):
    return [random.choices((0, 1), k=nr_cols)
            for _ in range(nr_rows)]

In [16]:
random.seed(1234)

In [17]:
world_map = create_map(5, 6)
world_map

[[1, 0, 0, 1, 1, 1],
 [1, 0, 1, 0, 0, 1],
 [0, 1, 1, 0, 0, 0],
 [0, 0, 1, 0, 1, 0],
 [1, 0, 1, 0, 1, 1]]

The problem is to find the islands.  In the map above, we have the following islands:
* {(0, 0), (1, 0)}
* {(0, 3), (0, 4), (0, 5), (1, 5)}
* {(1, 2), (2, 1), (2, 2), (3, 2), (4, 2)}
* {(3, 4), (4, 4), (4, 5)}
* {(4, 0)}

# Implementation

In [18]:
def find_islands(world_map):
    islands = []
    for row_nr, row in enumerate(world_map):
        for col_nr in range(len(row)):
            if world_map[row_nr][col_nr]:
                next_islands = []
                # A location can be added to 0, 1, or 2 islands.  If it
                # is added to 2 islands, it forms a bridge between them and
                # they should be merged.
                enlarged_islands = []
                for island in islands:
                    if (row_nr, col_nr - 1) in island or (row_nr - 1, col_nr) in island:
                        island.add((row_nr, col_nr))
                        enlarged_islands.append(island)
                    else:
                        next_islands.append(island)
                match len(enlarged_islands):
                    case 0:
                        # The location wasn't added ot any existing island, so it forms
                        # an island by itself.
                        next_islands.append({(row_nr, col_nr)})
                    case 1:
                        # The location was added to a single existing island.
                        next_islands.append(enlarged_islands[0])
                    case 2:
                        # The location was added to 2 islands, they have to be merged.
                        new_island = enlarged_islands[0].union(enlarged_islands[1])
                        next_islands.append(new_island)
                    case _:
                        warnings.warn("# error")
                islands = next_islands
    return islands            

In [19]:
world_map

[[1, 0, 0, 1, 1, 1],
 [1, 0, 1, 0, 0, 1],
 [0, 1, 1, 0, 0, 0],
 [0, 0, 1, 0, 1, 0],
 [1, 0, 1, 0, 1, 1]]

In [20]:
find_islands(world_map)

[{(0, 0), (1, 0)},
 {(0, 3), (0, 4), (0, 5), (1, 5)},
 {(4, 0)},
 {(1, 2), (2, 1), (2, 2), (3, 2), (4, 2)},
 {(3, 4), (4, 4), (4, 5)}]

# Verification

In [21]:
def is_on_map(location, world_map):
    nr_rows, nr_cols = len(world_map), len(world_map[0])
    return 0 <= location[0] < nr_rows and 0 <= location[1] < nr_cols

In [22]:
def get_neighbours(location, world_map):
    neighbours = []
    for delta in (-1, 1):
        neighbour = (location[0] + delta, location[1])
        if is_on_map(neighbour, world_map):
            neighbours.append(neighbour)
        neighbour = (location[0], location[1] + delta)
        if is_on_map(neighbour, world_map):
            neighbours.append(neighbour)
    return neighbours

In [23]:
def is_island(island, world_map):
    for location in island:
        if world_map[location[0]][location[1]] != 1:
            return False
        for neighbour in get_neighbours(location, world_map):
            if world_map[neighbour[0]][neighbour[1]] == 1 and neighbour not in island:
                return False
            if world_map[neighbour[0]][neighbour[1]] == 0 and neighbour in island:
                return False
    return True

In [30]:
def are_islands(islands, world_map):
    for island in islands:
        if not is_island(island, world_map):
            return False
    for i, island1 in enumerate(islands):
        for island2 in islands[i + 1:]:
            if island1.intersection(island2):
                return False
    nr_rows, nr_cols = len(world_map), len(world_map[0])
    all_land = {(i, j) for i, j in itertools.product(range(nr_rows), range(nr_cols))
                if world_map[i][j] == 1}
    all_island_land = set()
    for island in islands:
        all_island_land.update(island)
    return all_land == all_island_land

In [31]:
are_islands(islands, world_map)

True

In [32]:
for _ in range(100):
    world_map = create_map(20, 20)
    islands = find_islands(world_map)
    if not are_islands(islands, world_map):
        print(world_map)